In [1]:
import pandas as pd

# Check Inference Data Preprocessing

In [6]:
city = "Bogor"
time_series_model = "ARIMA"
lst_time_series_feature = [
    'median_length', 
    'median_delay', 
    'median_speed_kmh', 
    'median_regular_speed', 
    'median_delay_seconds'
]
dataset_path_template = "../data/processed/final_dataset_{}.parquet.gzip"
time_series_result_template = "../data/model_result/time_series/{}_{}_{}.parquet.gzip"

In [11]:
df = pd.read_parquet(dataset_path_template.format(city))
df = df[df['classification_split'] == 'test']
df

,time,street,level,median_length,median_delay,median_speed_kmh,median_regular_speed,median_delay_seconds,rain_intensity,holiday_gap,time_series_split,classification_split
61410,2022-08-31 00:00:00,Surya Kencana,0,1192.140229,10.148264,21.323593,20.414154,12.697850,0.0,-1,test,test
61411,2022-08-31 00:00:00,RE Martadinata,0,1221.311475,11.311475,27.311475,21.088608,11.129032,0.0,-1,test,test
61412,2022-08-31 00:00:00,Binamarga,0,1187.818389,9.843616,25.889055,22.174524,10.143151,0.0,-1,test,test
61413,2022-08-31 00:00:00,Jenderal Sudirman,0,1578.128999,9.626991,31.258333,22.820000,78.500000,0.0,-1,test,test
61414,2022-08-31 00:00:00,Taman Cimanggu Raya,0,1338.483162,9.893030,24.729622,22.604954,11.497557,0.0,-1,test,test
...,...,...,...,...,...,...,...,...,...,...,...,...
66833,2022-09-04 21:00:00,Cilebut Raya,0,1439.817561,11.573804,23.706937,19.270000,60.500000,0.0,-1,test,test
66834,2022-09-04 21:00:00,Semplak Raya,3,539.000000,152.000000,8.610000,19.470000,79.000000,0.0,-1,test,test
66835,2022-09-04 21:00:00,Pahlawan,0,1799.920448,9.599060,32.400000,19.705000,74.000000,0.0,-1,test,test
66836,2022-09-04 21:00:00,N9 KH Soleh Iskandar,3,442.000000,108.000000,11.540000,23.987499,76.000000,0.0,-1,test,test


In [42]:
time_series_pred = {}
for feature in lst_time_series_feature:
    time_series_pred[feature] = pd.read_parquet(
        time_series_result_template.format(time_series_model, city, feature)
    )

In [32]:
time_series_pred['median_delay']

,street,time,actual,pred
0,Kebon Pedes,2022-08-24 00:00:00,9.937105,25.285189
1,Kebon Pedes,2022-08-24 01:00:00,8.997109,28.294072
2,Kebon Pedes,2022-08-24 02:00:00,10.242862,30.332896
3,Kebon Pedes,2022-08-24 03:00:00,10.738484,31.714406
4,Kebon Pedes,2022-08-24 04:00:00,9.511229,32.650520
...,...,...,...,...
13151,N9 Ir Haji Juanda,2022-09-04 17:00:00,125.000000,68.196574
13152,N9 Ir Haji Juanda,2022-09-04 18:00:00,207.000000,68.196574
13153,N9 Ir Haji Juanda,2022-09-04 19:00:00,89.000000,68.196574
13154,N9 Ir Haji Juanda,2022-09-04 20:00:00,9.056381,68.196574


In [23]:
tmp = df.drop(['median_delay'], axis=1)
ts = time_series_pred['median_delay']
ts.drop(['actual'], axis=1, inplace=True)
ts.rename(columns={
    'pred': 'median_delay'
}, inplace=True)
tmp.join(
    ts.set_index(['time', 'street']),
    on=['time', 'street'],
    how='inner'
)

,time,street,level,median_length,median_speed_kmh,median_regular_speed,median_delay_seconds,rain_intensity,holiday_gap,time_series_split,classification_split,median_delay
61410,2022-08-31 00:00:00,Surya Kencana,0,1192.140229,21.323593,20.414154,12.697850,0.0,-1,test,test,119.705286
61411,2022-08-31 00:00:00,RE Martadinata,0,1221.311475,27.311475,21.088608,11.129032,0.0,-1,test,test,56.097950
61412,2022-08-31 00:00:00,Binamarga,0,1187.818389,25.889055,22.174524,10.143151,0.0,-1,test,test,46.039784
61413,2022-08-31 00:00:00,Jenderal Sudirman,0,1578.128999,31.258333,22.820000,78.500000,0.0,-1,test,test,44.320485
61414,2022-08-31 00:00:00,Taman Cimanggu Raya,0,1338.483162,24.729622,22.604954,11.497557,0.0,-1,test,test,54.992715
...,...,...,...,...,...,...,...,...,...,...,...,...
66833,2022-09-04 21:00:00,Cilebut Raya,0,1439.817561,23.706937,19.270000,60.500000,0.0,-1,test,test,55.620497
66834,2022-09-04 21:00:00,Semplak Raya,3,539.000000,8.610000,19.470000,79.000000,0.0,-1,test,test,99.054059
66835,2022-09-04 21:00:00,Pahlawan,0,1799.920448,32.400000,19.705000,74.000000,0.0,-1,test,test,75.917544
66836,2022-09-04 21:00:00,N9 KH Soleh Iskandar,3,442.000000,11.540000,23.987499,76.000000,0.0,-1,test,test,88.606043


In [28]:
def change_feature(dataset, time_series):
    result = dataset.copy()
    result.drop(lst_time_series_feature, axis=1, inplace=True)
    for feature in lst_time_series_feature:
        data = time_series[feature]
        data.drop(['actual'], axis=1, inplace=True)
        data.rename(columns={
            'pred': feature
        }, inplace=True)
        result = result.join(
            data.set_index(['time', 'street']),
            on=['time', 'street'],
            how='inner',
        )
    return result

In [43]:
a = change_feature(df, time_series_pred)
a.replace(-1, 8, inplace=True)
a

,time,street,level,rain_intensity,holiday_gap,time_series_split,classification_split,median_length,median_delay,median_speed_kmh,median_regular_speed,median_delay_seconds
61410,2022-08-31 00:00:00,Surya Kencana,0,0.0,8,test,test,1083.263515,119.705286,24.143504,18.360732,45.344944
61411,2022-08-31 00:00:00,RE Martadinata,0,0.0,8,test,test,947.237458,56.097950,19.098030,21.914765,11.568716
61412,2022-08-31 00:00:00,Binamarga,0,0.0,8,test,test,943.254476,46.039784,21.045616,21.906465,12.135207
61413,2022-08-31 00:00:00,Jenderal Sudirman,0,0.0,8,test,test,1315.302525,44.320485,30.498454,22.302701,79.963886
61414,2022-08-31 00:00:00,Taman Cimanggu Raya,0,0.0,8,test,test,1118.102850,54.992715,20.371296,22.729501,11.347078
...,...,...,...,...,...,...,...,...,...,...,...,...
66833,2022-09-04 21:00:00,Cilebut Raya,0,0.0,8,test,test,1207.883194,55.620497,19.964025,19.613838,59.001235
66834,2022-09-04 21:00:00,Semplak Raya,3,0.0,8,test,test,1293.624576,99.054059,20.261813,20.637772,76.863704
66835,2022-09-04 21:00:00,Pahlawan,0,0.0,8,test,test,1264.702578,75.917544,23.475143,19.840296,71.433882
66836,2022-09-04 21:00:00,N9 KH Soleh Iskandar,3,0.0,8,test,test,1414.831178,88.606043,31.903925,23.681871,81.046347


In [38]:
tmp = time_series_pred['median_length']
tmp[
    (tmp['street'] == 'Binamarga') & \
        (tmp['time'] == '2022-08-31')
]

,street,time,median_length
8462,Binamarga,2022-08-31,943.254476
